# Work in Progress/Add in Future:
1. Scrape data for min. 1 week
2. Change cities

In [1]:
import os
import sys
from datetime import date
import glob
import numpy as np
import pandas as pd

In [2]:
sys.path.insert(0, os.path.abspath('..\modules'))

In [3]:
import pricemate as pm

In [4]:
# scrape data using pricemate module
df = pm.multiple_days_df(0,10)

# add scrape_date
df['scrape_date'] = date.today()

Trying to download new driver from http://chromedriver.storage.googleapis.com/79.0.3945.36/chromedriver_win32.zip
Unpack archive C:\Users\Zenbook\.wdm\drivers\chromedriver\79.0.3945.36\win32\chromedriver.zip

Looking for [chromedriver 79.0.3945.36 win32] driver in cache 
File found in cache by path [C:\Users\Zenbook\.wdm\drivers\chromedriver\79.0.3945.36\win32\chromedriver.exe]

Looking for [chromedriver 79.0.3945.36 win32] driver in cache 
File found in cache by path [C:\Users\Zenbook\.wdm\drivers\chromedriver\79.0.3945.36\win32\chromedriver.exe]

Looking for [chromedriver 79.0.3945.36 win32] driver in cache 
File found in cache by path [C:\Users\Zenbook\.wdm\drivers\chromedriver\79.0.3945.36\win32\chromedriver.exe]

Looking for [chromedriver 79.0.3945.36 win32] driver in cache 
File found in cache by path [C:\Users\Zenbook\.wdm\drivers\chromedriver\79.0.3945.36\win32\chromedriver.exe]

Looking for [chromedriver 79.0.3945.36 win32] driver in cache 
File found in cache by path [C:\User

In [42]:
# import latest scrape data, if any
# check latest scrape data date
files = glob.glob('..\scrape-csv\*.csv')
lastdate = max([i[-12:-4] for i in scrapefiles])
lastscrape = scrapefiles[scrapedates.index(latestscrapedate)]
lastdf = pd.read_csv(lastscrape)
lastdf.head()

,arrival_date,arrive_time,depart_date,depart_time,price,seats,tclass,title,scrape_date
0,2020-01-09,08:46,2020-01-09,05:30,110000.0,99,ekonomi (Subclass S),argo parahyangan 36,2020-01-05
1,2020-01-09,09:49,2020-01-09,06:15,110000.0,99,ekonomi (Subclass S),argo parahyangan 38,2020-01-05
2,2020-01-09,11:19,2020-01-09,07:50,110000.0,99,ekonomi (Subclass C),pangandaran 176,2020-01-05
3,2020-01-09,12:13,2020-01-09,08:55,110000.0,99,ekonomi (Subclass S),argo parahyangan 40,2020-01-05
4,2020-01-09,14:09,2020-01-09,10:45,110000.0,99,ekonomi (Subclass S),argo parahyangan 42,2020-01-05


In [41]:
# combine with today's scrape
todaymarker = date.today().strftime('%Y%m%d')
if lastdate < todaymarker:
    df.append(lastdf)
else:
    print('Last scrape date is not before today')

Last scrape date is not before today


In [35]:
# export combined scrape output to new csv
datemarker = date.today().strftime('%Y%m%d')
df.to_csv('..\scrape-csv\gmr-bdg-' + datemarker + '.csv', index=False)

In [43]:
# standard procedure: make a copy of the original data and examine head
df_raw = df

df.head()

,arrival_date,arrive_time,depart_date,depart_time,price,seats,tclass,title,scrape_date
0,2020-01-09,08:46,2020-01-09,05:30,110000,99,ekonomi (Subclass S),argo parahyangan 36,2020-01-05
1,2020-01-09,09:49,2020-01-09,06:15,110000,99,ekonomi (Subclass S),argo parahyangan 38,2020-01-05
2,2020-01-09,11:19,2020-01-09,07:50,110000,99,ekonomi (Subclass C),pangandaran 176,2020-01-05
3,2020-01-09,12:13,2020-01-09,08:55,110000,99,ekonomi (Subclass S),argo parahyangan 40,2020-01-05
4,2020-01-09,14:09,2020-01-09,10:45,110000,99,ekonomi (Subclass S),argo parahyangan 42,2020-01-05


In [45]:
df.describe()

,arrival_date,arrive_time,depart_date,depart_time,price,seats,tclass,title,scrape_date
count,673,673,673,673,673,673,673,673,673
unique,7,18,6,18,14,8,10,28,1
top,2020-01-12,20:29,2020-01-12,17:10,,0,eksekutif (Subclass I),mutiara selatan 104,2020-01-05
freq,116,60,116,60,419,419,142,60,673


In [49]:
df.dtypes
# all data types seem to be objects

arrival_date    object
arrive_time     object
depart_date     object
depart_time     object
price           object
seats           object
tclass          object
title           object
scrape_date     object
dtype: object

## Diagnostic assumptions
- Arrival date >= depart date
- depart date >= scrape date
- seats >= 0


In [ ]:
df['price'] - df['price']

## Data Preprocessing
- data types: dates = dates, times = times, price & seats = integer

In [50]:
df[['price', 'seats']].astype('int32')

KeyError: ('price', 'seats')

## Exploratory questions & assumptions
1. show earliest available ekonomi train
2. 

In [ ]:
# filter available tickets only
df_avail = df